# Twithischia
In which we implement [@GalacticFurball's](https://twitter.com/GalacticFurball)
[Twitter steganography](https://twitter.com/GalacticFurball/status/1439259660658241539)
clumsily with [Numpy](https://numpy.org/) instead of elegantly with
[ImageMagick](https://github.com/discatte/tweetdoom). (One also might get it going in the browser
with [Pyodide](https://pyodide.org/en/stable/))

We hide .zip files in the lower 4 bits of Twitter images, providing Twitter has `b0rked` them *isufficiently*.

*Gentle reader*, click "Run All" from the "Cell" or "Run" menu...

In [ ]:
from io import BytesIO
from itertools import starmap
import os
import urllib3
import zipfile

import imageio
from IPython.display import display
import ipywidgets as widgets
import numpy as np

In [ ]:
def pull_image(url):
    pool = urllib3.PoolManager()
    try:
        resp = pool.request('GET', url, preload_content=False).data
        return imageio.imread(resp)
    except:
        return None

By some miracle, [np.ravel()](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html) gets the image
the right way up. Then, we just need to combine the low and high nibbles from the original .zip file,

In [ ]:
def raw_bytes(img):
    nibbles = img.ravel() & 15
    return 16 * nibbles[0::2] + nibbles[1::2]

Many zip implentations don't ike zero-padding, thus we look for the 
[End_of_central_directory_record_(EOCD)](https://en.wikipedia.org/wiki/ZIP_(file_format)#End_of_central_directory_record_(EOCD)), `0x06054b50`. 20 bytes in from that is a little-endian word telling us how many comment bytes follow.

In [ ]:
__eocdr__ = np.array([80, 75, 5, 6])

def find_eocdr(arr):
    # There's got to be a better way...
    for i in range(len(arr)-4, -1, -1):
        if (arr[i:i+4] == __eocdr__).all():
            return i
    return None


def trim_zip(arr):
    end = find_eocdr(arr)
    if not end:
        return None
    com_len = arr[end+20] + arr[end+21] << 8
    return arr[:end+22+com_len]


def test_zip(arr):
    try:
        zp = zipfile.ZipFile(BytesIO(arr.tobytes()))
    except:
        return None
    
    return [(f.filename, f.file_size) for f in zp.filelist]

Various bodging with [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/):

In [ ]:
urls_box = widgets.VBox([widgets.Text(placeholder='enter the URL of a .png')])
add_button = widgets.Button(description='add url')
get_button = widgets.Button(description='get .zip')
but_box = widgets.HBox([add_button, get_button])
download = widgets.HTML()
root_box = widgets.VBox([urls_box, but_box, download])

In [ ]:
def new_url(_):
    urls_box.children = tuple(urls_box.children
        +(widgets.Text(placeholder='enter the URL of a .png'),))

In [ ]:
def get_zip(_):
    images = (pull_image(t.value) for t in urls_box.children)
    zip_bytes =  np.concatenate([raw_bytes(img)
        for img in images if not img is None])
    clean_zip = trim_zip(zip_bytes)
    
    content = test_zip(clean_zip)
    
    if content:
        table = '<table>{}</table>'.format('\n'.join(
            starmap('<tr><td>{}</td><td>{}</td></tr>'.format, content)))
        with open('twithischia.zip', 'wb') as f:
            f.write(clean_zip.tobytes())
    else:
        download.value = '<h3>Broken URL, image or zipfile</h3>'

## Decoding Images:

Some manner of UI should appear below...

*Gentle reader*, you might want to paste in these URLs if you want to play Doom:

`https://pbs.twimg.com/media/E_lG3ujVUAEKqWN?format=png&name=900x900`

`https://pbs.twimg.com/media/E_lG4SVVcAUK9Rq?format=png&name=900x900`

`https://pbs.twimg.com/media/E_lG4_LVEAAclU2?format=png&name=900x900`

In [ ]:
add_button.on_click(new_url)
get_button.on_click(get_zip)
display(root_box)